In [9]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

test_data = pd.read_csv("Test_data.csv")
test_data

train_data = pd.read_csv("Training_data.csv")
train_data

carparks = np.load('carpark_numbers_cbd_area.npy')

In [10]:
train_data["update_datetime"] = pd.to_datetime(train_data["update_datetime"])
train_data["month"] = train_data["update_datetime"].dt.month

sept_data = train_data.groupby("month").get_group(9)

In [12]:
all_data = pd.DataFrame()

for carpark in carparks:
    
    print("Getting sept data agg for ", carpark)
    sept_data_1 = sept_data[sept_data['carpark_number'] == carpark]
    data = pd.DataFrame()
    sept_data_agg = pd.DataFrame()

    sept_data_daily = sept_data_1.groupby("day", as_index = False)
    hours = []
    for day in range(1, len(sept_data_daily.groups.keys()) + 1):
        for hour in range(0, 24):
            hours.append(24*(day-1) + hour)

    for day in sept_data_daily.groups.keys():
        day_data = sept_data_daily.get_group(day)
        hourly_data_day = day_data.groupby('hour', as_index = False)['percent_lots_available'].mean()

        hourly_data = []
        hour = []
        for i in range(0, 24):
            hour.append(i)
            value = np.NaN if hourly_data_day[hourly_data_day['hour'] == i]['percent_lots_available'].empty \
            else hourly_data_day[hourly_data_day['hour'] == i]['percent_lots_available'].iloc[0]
            hourly_data.append(value)

        data['hour'] = hour
        data['percent_lots_available'] = hourly_data
        data['day'] = day
        sept_data_agg = sept_data_agg.append(data, ignore_index=True)
        
    sept_data_agg['carpark_number'] = carpark

    sept_data_agg.interpolate(method='linear', limit_direction='both', inplace = True)

    sept_data_agg['lag_1'] = sept_data_agg['percent_lots_available'].shift(1)
    sept_data_agg['lag_168'] = sept_data_agg['percent_lots_available'].shift(168)
    
    
    print(sept_data_agg)
    
    print("Getting oct data agg for ", carpark)
    
    test_data_1 = test_data[test_data['carpark_number'] == carpark]
    data = pd.DataFrame()
    test_data_agg = pd.DataFrame()

    test_data_daily = test_data_1.groupby("day", as_index = False)
    hours = []
    for day in range(1, len(test_data_daily.groups.keys()) + 1):
        for hour in range(0, 24):
            hours.append(24*(day-1) + hour)

    for day in test_data_daily.groups.keys():
        day_data = test_data_daily.get_group(day)
        hourly_data_day = day_data.groupby('hour', as_index = False)['percent_lots_available'].mean()

        hourly_data = []
        hour = []
        for i in range(0, 24):
            hour.append(i)
            value = np.NaN if hourly_data_day[hourly_data_day['hour'] == i]['percent_lots_available'].empty \
            else hourly_data_day[hourly_data_day['hour'] == i]['percent_lots_available'].iloc[0]
            hourly_data.append(value)
        
        data['carpark_number'] = carpark
        data['hour'] = hour
        data['percent_lots_available'] = hourly_data
        data['day'] = day
        test_data_agg = test_data_agg.append(data, ignore_index=True)
        
    test_data_agg['carpark_number'] = carpark

    test_data_agg.interpolate(method='linear', limit_direction='both', inplace = True)

    test_data_agg['lag_1'] = test_data_agg['percent_lots_available'].shift(1)
    test_data_agg['lag_168'] = test_data_agg['percent_lots_available'].shift(168)

    test_data_agg['lag_1'].iloc[0] = sept_data_agg.tail(1)['percent_lots_available']
    test_data_agg['lag_168'].iloc[:168] = sept_data_agg.tail(168)['percent_lots_available']
    
    print(test_data_agg)
    
    all_data = all_data.append(test_data_agg)
    
all_data


Getting sept data agg for  AR5M
     hour  percent_lots_available  day carpark_number      lag_1    lag_168
0       0               52.505967    1           AR5M        NaN        NaN
1       1               52.505967    1           AR5M  52.505967        NaN
2       2               54.256165    1           AR5M  52.505967        NaN
3       3               57.756563    1           AR5M  54.256165        NaN
4       4               57.756563    1           AR5M  57.756563        NaN
..    ...                     ...  ...            ...        ...        ...
715    19               54.892601   30           AR5M  54.892601  54.653938
716    20               54.892601   30           AR5M  54.892601  54.653938
717    21               54.892601   30           AR5M  54.892601  54.653938
718    22               54.892601   30           AR5M  54.892601  54.653938
719    23               54.892601   30           AR5M  54.892601  54.653938

[720 rows x 6 columns]
Getting oct data agg for  AR5M
 

    carpark_number  hour  percent_lots_available  day      lag_1    lag_168
0              BM1     0               13.888889    1  22.500000  20.000000
1              BM1     1               13.888889    1  13.888889  12.500000
2              BM1     2               18.333333    1  13.888889   6.111111
3              BM1     3               26.666667    1  18.333333  15.000000
4              BM1     4               30.000000    1  26.666667  18.888889
..             ...   ...                     ...  ...        ...        ...
739            BM1    19               12.083333   31  21.666667   0.000000
740            BM1    20                5.000000   31  12.083333  10.000000
741            BM1    21               27.777778   31   5.000000  17.777778
742            BM1    22               20.000000   31  27.777778  22.777778
743            BM1    23               20.833333   31  20.000000  29.444444

[744 rows x 6 columns]
Getting sept data agg for  BM10
     hour  percent_lots_availabl

     hour  percent_lots_available  day carpark_number      lag_1    lag_168
0       0               42.138365    1            BM2        NaN        NaN
1       1               43.396226    1            BM2  42.138365        NaN
2       2               45.911950    1            BM2  43.396226        NaN
3       3               49.685535    1            BM2  45.911950        NaN
4       4               50.943396    1            BM2  49.685535        NaN
..    ...                     ...  ...            ...        ...        ...
715    19               29.245283   30            BM2  29.559748   2.515723
716    20               35.220126   30            BM2  29.245283  14.465409
717    21               41.509434   30            BM2  35.220126  30.188679
718    22               52.201258   30            BM2  41.509434  37.735849
719    23               53.773585   30            BM2  52.201258  40.094340

[720 rows x 6 columns]
Getting oct data agg for  BM2
    carpark_number  hour  percent_

    carpark_number  hour  percent_lots_available  day      lag_1    lag_168
0             BM29     0               78.694158    1  70.618557  80.927835
1             BM29     1               82.817869    1  78.694158  83.762887
2             BM29     2               84.192440    1  82.817869  85.567010
3             BM29     3               81.786942    1  84.192440  85.223368
4             BM29     4               83.505155    1  81.786942  85.567010
..             ...   ...                     ...  ...        ...        ...
739           BM29    19               60.824742   31  51.546392  61.512027
740           BM29    20               73.195876   31  60.824742  73.539519
741           BM29    21               79.725086   31  73.195876  75.257732
742           BM29    22               83.505155   31  79.725086  78.006873
743           BM29    23               85.051546   31  83.505155  86.597938

[744 rows x 6 columns]
Getting sept data agg for  BM3
     hour  percent_lots_available

     hour  percent_lots_available  day carpark_number  lag_1    lag_168
0       0                     0.0    1            BM5    NaN        NaN
1       1                     0.0    1            BM5    0.0        NaN
2       2                     0.0    1            BM5    0.0        NaN
3       3                     0.0    1            BM5    0.0        NaN
4       4                     0.0    1            BM5    0.0        NaN
..    ...                     ...  ...            ...    ...        ...
715    19                     0.0   30            BM5    0.0  39.130435
716    20                     0.0   30            BM5    0.0  39.130435
717    21                     0.0   30            BM5    0.0  39.130435
718    22                     0.0   30            BM5    0.0  39.130435
719    23                     0.0   30            BM5    0.0  30.434783

[720 rows x 6 columns]
Getting oct data agg for  BM5
    carpark_number  hour  percent_lots_available  day  lag_1    lag_168
0         

...

     hour  percent_lots_available  day carpark_number      lag_1    lag_168
0       0               77.250000    1           TBM6        NaN        NaN
1       1               76.833333    1           TBM6  77.250000        NaN
2       2               76.750000    1           TBM6  76.833333        NaN
3       3               76.250000    1           TBM6  76.750000        NaN
4       4               76.000000    1           TBM6  76.250000        NaN
..    ...                     ...  ...            ...        ...        ...
595    19               85.937500   30           TBM6  88.750000  83.250000
596    20               85.666667   30           TBM6  85.937500  83.250000
597    21               85.083333   30           TBM6  85.666667  80.500000
598    22               83.833333   30           TBM6  85.083333  79.166667
599    23               81.875000   30           TBM6  83.833333  78.125000

[600 rows x 6 columns]
Getting oct data agg for  TBM6
    carpark_number  hour  percent

    carpark_number  hour  percent_lots_available  day      lag_1    lag_168
0              TE1     0               58.888889    1  58.888889  26.666667
1              TE1     1               58.888889    1  58.888889  35.555556
2              TE1     2               58.888889    1  58.888889  37.916667
3              TE1     3               58.888889    1  58.888889  35.555556
4              TE1     4               58.888889    1  58.888889  35.555556
..             ...   ...                     ...  ...        ...        ...
739            TE1    19               58.888889   31  58.888889  58.888889
740            TE1    20               58.888889   31  58.888889  58.888889
741            TE1    21               58.888889   31  58.888889  58.888889
742            TE1    22               58.888889   31  58.888889  58.888889
743            TE1    23               58.888889   31  58.888889  58.888889

[744 rows x 6 columns]
Getting sept data agg for  TE13
     hour  percent_lots_availabl

     hour  percent_lots_available  day carpark_number  lag_1  lag_168
0       0                     0.0    1            TE3    NaN      NaN
1       1                     0.0    1            TE3    0.0      NaN
2       2                     0.0    1            TE3    0.0      NaN
3       3                     0.0    1            TE3    0.0      NaN
4       4                     0.0    1            TE3    0.0      NaN
..    ...                     ...  ...            ...    ...      ...
715    19                     0.0   30            TE3    0.0      0.0
716    20                     0.0   30            TE3    0.0      0.0
717    21                     0.0   30            TE3    0.0      0.0
718    22                     0.0   30            TE3    0.0      0.0
719    23                     0.0   30            TE3    0.0      0.0

[720 rows x 6 columns]
Getting oct data agg for  TE3
    carpark_number  hour  percent_lots_available  day  lag_1  lag_168
0              TE3     0            

    carpark_number  hour  percent_lots_available  day    lag_1  lag_168
0              YHS     0               32.812500    1  32.8125  46.8750
1              YHS     1               32.812500    1  32.8125  46.8750
2              YHS     2               32.812500    1  32.8125  46.8750
3              YHS     3               32.812500    1  32.8125  46.8750
4              YHS     4               32.812500    1  32.8125  46.8750
..             ...   ...                     ...  ...      ...      ...
739            YHS    19               32.812500   31  32.8125  32.8125
740            YHS    20               32.812500   31  32.8125  32.8125
741            YHS    21               32.812500   31  32.8125  32.8125
742            YHS    22               32.812500   31  32.8125  32.8125
743            YHS    23               47.935606   31  32.8125  32.8125

[744 rows x 6 columns]


,carpark_number,hour,percent_lots_available,day,lag_1,lag_168
0,AR5M,0,54.892601,1,54.892601,54.653938
1,AR5M,1,54.892601,1,54.892601,54.653938
2,AR5M,2,54.892601,1,54.892601,54.653938
3,AR5M,3,56.085919,1,54.892601,54.653938
4,AR5M,4,56.085919,1,56.085919,54.653938
...,...,...,...,...,...,...
739,YHS,19,32.812500,31,32.812500,32.812500
740,YHS,20,32.812500,31,32.812500,32.812500
741,YHS,21,32.812500,31,32.812500,32.812500
742,YHS,22,32.812500,31,32.812500,32.812500


In [13]:
all_data.to_csv("oct_data_with_lags.csv", index = False)